# Run basic calculations using Workgraph

## Aim

This notebook sets up two basic workgraphs to run calculations, to show how WorkGraph interacts with our custom calculations

## Single Point Calculation

### Setup

The initial setup is very similar to the other tutorials, such as `singlepoint.ipynb`, which goes into more detail about what each step is doing

Load the aiida profile:

In [ ]:
# Load profile
from aiida import load_profile
load_profile()

Get the structure, model and load the code:

In [ ]:
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read

#structure = StructureData(ase=read("Structures/qmof-ffeef76.cif"))
structure = StructureData(ase=bulk("NaCl", "rocksalt", 5.63))

In [ ]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

In [ ]:
from aiida.orm import load_code
code = load_code("janus@localhost")

Inputs should include the model, code, metadata, and any other keyword arguments expected by the calculation we are running:

In [ ]:
from aiida.orm import Str
inputs = {
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    "code": code,
    "arch": model.architecture,
    "precision": Str("float64"),
    "struct": structure,
    "model": model,
    "device": Str("cpu"),
}

We must also choose the calculation to perform:

In [ ]:
from aiida.plugins import CalculationFactory
singlepointCalc = CalculationFactory("mlip.sp")

### Creating and running a workgraph

We can now create a workgraph by first loading WorkGraph and giving it a name  (`"SinglepointWorkGraph"` in this example). We then create a task for our calculation and assign this task a name

In [ ]:
from aiida_workgraph import WorkGraph

wg = WorkGraph("SinglepointWorkGraph")

sp_calc = wg.add_task(
    singlepointCalc,
    name="sp_calc",
    **inputs
)

In [ ]:
wg

We can visual the tasks of the Workgraph and run the tasks

In [ ]:
wg.tasks.sp_calc

In [ ]:
wg.run()

The graph can be visualised

In [ ]:
from aiida_workgraph.utils import generate_node_graph

generate_node_graph(wg.pk)

We can then check the output to ensure we are getting the correct output. Here, we check the `xyz_output`, which should return a `SinglefileData` object

In [ ]:
type(wg.tasks.sp_calc.outputs.xyz_output.value)

We can also print the outputs of the calculation

In [ ]:
wg.tasks.sp_calc.outputs.results_dict.value.get_dict()

We can also use verdi to interact with the calcaltion and see the output (you may need to click `view scrollable element` if you are running the command in a notebook to see the mose recent tasks)

In [ ]:
! verdi process list -a

Using the pk, we can print the output from verdi

In [ ]:
! verdi calcjob res [pk]  

## Geometry Optimisation

The calculations can be setup with WorkGraph very similarly in most cases. In this example we will setup a geometry optimisation calculation. First we setup the the inputs and load the calculation

In [ ]:
from aiida.orm import Str, Float, Bool
inputs = {
        "code": code,
        "model": model,
        "struct": structure,
        "arch": Str(model.architecture),
        "precision": Str("float64"),
        "device": Str("cpu"),
        "fmax": Float(0.1), 
        "opt_cell_lengths": Bool(False), 
        "opt_cell_fully": Bool(True), 
        "metadata": {"options": {"resources": {"num_machines": 1}}},
    }

In [ ]:
from aiida.plugins import CalculationFactory
geomoptCalc = CalculationFactory("mlip.opt")

We can then start building the WorkGraph similar to before

In [ ]:
from aiida_workgraph import WorkGraph

wg = WorkGraph("Geometry Optimisation")

geomopt_calc = wg.add_task(
    geomoptCalc,
    name="geomopt_calc",
)

This time if we want to pass the inputs/outputs directly to the workgraph we can define them. 
As we are passing in the inputs as kwargs we have to pass the inputs directly to the task

In [ ]:
wg.tasks.geomopt_calc.set(
    {**inputs}
)

In [ ]:
wg.outputs.results = wg.tasks.geomopt_calc.outputs.results_dict
wg.outputs.results_file = wg.tasks.geomopt_calc.outputs.xyz_output

Now we can visually see the outputs from the `geomopt_calc` task being passed into the workgraph outputs:

In [ ]:
wg

Run the workgraph

In [ ]:
wg.run()

We can access the outputs as before, but we can also use the `wg.outputs` that we set up

In [ ]:
type(wg.outputs.results_file.value)

In [ ]:
wg.outputs.results.value.get_dict()

We can also access the outputs from verdi

In [ ]:
! verdi process list -a

In [ ]:
! verdi calcjob res [pk]